In [66]:
%matplotlib inline

import numpy as np
import requests
import re
import pickle
import pandas as pd
import matplotlib.pyplot as pl

In [67]:
cols = ['Project Number', 'Institution', 'University', 'Approved Amount']
na_values = ['data not included in P3', 'Nicht zuteilbar - NA']

dtypes = {
        'Approved Amount': np.float64
}

raw = pd.read_csv('P3_GrantExport.csv', sep = ';', na_values=na_values, index_col='Project Number', dtype=dtypes, usecols=cols)
df = raw.dropna()

In [68]:
df.sample(10)

,Institution,University,Approved Amount
Project Number,,,
143467,Schweizerischer Erdbebendienst ETH Zürich,ETH Zürich - ETHZ,250516.0
1719,Faculté de Théologie Université de Fribourg,Université de Fribourg - FR,126567.0
3056,EPFL - SB - MATHAA,EPF Lausanne - EPFL,85636.0
29813,Institut de Mathématiques Université de Lausanne,Université de Lausanne - LA,210260.0
28178,Institut d'Etudes sociales INTEREC Secteur Ser...,Université de Genève - GE,116034.0
129577,Deutsches Seminar Universität Basel,Universität Basel - BS,508766.0
110310,Biochemisches Institut ETH Hönggerberg,ETH Zürich - ETHZ,258051.0
47162,Centre Universitaire d'Informatique Université...,Université de Genève - GE,221549.0
50185,Eidg. Materialprüfungs- und Versuchsanstalt Ab...,Eidg. Material und Prüfungsanstalt - EMPA,160545.0


In [69]:
df.index.is_unique

True

In [70]:
with_canton = df.copy()

word_to_canton = {
    'bern': 'BE',
    'lausanne': 'VD',
    'genève': 'GE',
    'geneva': 'GE',
    'luzern': 'LU',
    'zürich': 'ZH',
    'lugano': 'TI',
    'basel': 'BS',
    'vaud': 'VD',
    'fribourg': 'FR'
}

def guess_canton(text):
    lower = text.lower()
    for word in word_to_canton:
        if word in lower:
            return word_to_canton[word]
        
    return None


def ex_canton(text, axis):
    guess = guess_canton(text)
    if guess:
        return guess
    
    res = text.split('-')
    if len(res) < 2: 
        return text
    else:
        return res[1]
    
def ex_uni(text, axis):
    res = text.split('-')
    if len(res) < 2:
        return text.strip()
    else:
        return res[0].strip()

with_canton['Canton'] = with_canton['University'].apply(ex_canton, axis=1)
with_canton['University'] = with_canton['University'].apply(ex_uni, axis=1)

with_canton.sample(10)

,Institution,University,Approved Amount,Canton
Project Number,,,,
101232,Abteilung für Allgemeine Innere Medizin Medizi...,Universität Basel,204316.0,BS
132458,Laboratoire de nanostructures superficielles E...,EPF Lausanne,121833.0,VD
33422,Département des sciences de la Terre Universit...,Université de Genève,153790.0,GE
100868,Unité d'Investigations Anesthésiologiques Serv...,Université de Genève,127360.0,GE
133887,Entscheidungs- & Verhaltensspieltheorie D-GESS...,ETH Zürich,145000.0,ZH
27907,Geographisches Institut Universität Bern,Universität Bern,183322.0,BE
9083,Dépt Microbiologie et Médecine Moléculaire Fac...,Université de Genève,370541.0,GE
117972,Institut für Kooperationsforschung und -entwic...,Fachhochschule Nordwestschweiz (ohne PH),198808.0,FHNW
104231,Centre de recherche en physique des plasmas Pa...,EPF Lausanne,84518.0,VD


In [71]:
grouped = with_canton.groupby(['Canton', 'University']).sum().reset_index()
len(grouped)

76

In [72]:
cantons = pd.read_csv('cantons.csv', header=None)
cantons = cantons[0].values.tolist()

print(cantons)

['ZH', 'BE', 'LU', 'UR', 'SZ', 'OW', 'NW', 'GL', 'ZG', 'FR', 'SO', 'BS', 'BL', 'SH', 'AR', 'AI', 'SG', 'GR', 'AG', 'TG', 'TI', 'VD', 'VS', 'NE', 'GE', 'JU', 'CH']


In [73]:
def is_known_canton(x, axis):
    return x.strip() in cantons

wc = grouped.copy()
wc['IsCanton'] = wc['Canton'].apply(is_known_canton, axis=1)

In [74]:
len(wc[wc['IsCanton'] == False])

55

In [75]:
wc[wc['IsCanton'] == False]

,Canton,University,Approved Amount,IsCanton
0,AGS,Forschungsanstalten Agroscope,3.311572e+07,False
1,AORI,AO Research Institute,3.435621e+06,False
2,ASP,Pädag. Hochschule Tessin (Teilschule SUPSI),1.593170e+05,False
3,ASPIT,Weitere Spitäler,1.074981e+07,False
4,BITG,Biotechnologie Institut Thurgau,2.492535e+06,False
5,CREALP,Centre de rech. sur l'environnement alpin,1.567678e+06,False
6,CSEM,Swiss Center for Electronics and Microtech.,1.806825e+07,False
7,EAWAG,Eidg. Anstalt für Wasserversorgung,7.397585e+07,False
8,EHB,Eidg. Hochschulinstitut für Berufsbildung,2.086572e+06,False
9,EMPA,Eidg. Material und Prüfungsanstalt,5.793069e+07,False


In [76]:
len(wc[wc['IsCanton'] == False])

55

In [77]:
params = {
    'username': 'ada_drs',
    'country': 'CH',
    'type': 'json'
}

def geoname_query(q):
    params['q'] = q
    # print('Searching for %s...' % q)
    return requests.get('http://api.geonames.org/search', params)

def search_by(col):
    for i in wc[wc['IsCanton'] == False].index:
        row = wc.iloc[i]
        res = geoname_query(row[col].strip())
        json = res.json()

        if json['totalResultsCount'] > 0:
            canton = json['geonames'][0]['adminCode1']
            print('=> Found ' + canton)
            wc.set_value(i,'Canton', canton)
            
search_by('University')
search_by('Canton')

=> Found SG
Canton                                   SG
University         Kantonsspital St. Gallen
Approved Amount                 1.50341e+07
IsCanton                              False
Name: 33, dtype: object
=> Found AG
Canton                                                      AG
University         Pädag. Hochschule Tessin (Teilschule SUPSI)
Approved Amount                                         159317
IsCanton                                                 False
Name: 2, dtype: object
=> Found ZH
Canton                                             ZH
University         Eidg. Anstalt für Wasserversorgung
Approved Amount                           7.39759e+07
IsCanton                                        False
Name: 7, dtype: object
=> Found ZH
Canton                                             ZH
University         Eidg. Material und Prüfungsanstalt
Approved Amount                           5.79307e+07
IsCanton                                        False
Name: 9, dtype: object

In [78]:
wc_geonames = wc.copy()
wc_geonames['IsCanton'] = wc_geonames['Canton'].apply(is_known_canton, axis=1)

In [79]:
len(wc_geonames[wc_geonames['IsCanton'] == False])

46

In [81]:
wc_geonames[wc_geonames['IsCanton'] == False]

,Canton,University,Approved Amount,IsCanton
0,AGS,Forschungsanstalten Agroscope,3.311572e+07,False
1,AORI,AO Research Institute,3.435621e+06,False
3,ASPIT,Weitere Spitäler,1.074981e+07,False
4,BITG,Biotechnologie Institut Thurgau,2.492535e+06,False
5,CREALP,Centre de rech. sur l'environnement alpin,1.567678e+06,False
6,CSEM,Swiss Center for Electronics and Microtech.,1.806825e+07,False
8,EHB,Eidg. Hochschulinstitut für Berufsbildung,2.086572e+06,False
10,EOC,Ente Ospedaliero Cantonale,5.067172e+06,False
11,FFHS,Fernfachhochschule Schweiz (Mitglied SUPSI),1.200000e+04,False
12,FHKD,Fachhochschule Kalaidos,1.090280e+06,False
